### 

## Biblotecas

In [1]:
import csv
import json
import pandas as pd
from nltk.probability import FreqDist
from pprint import pprint
import nltk
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from string import punctuation
from textblob.classifiers import NaiveBayesClassifier
import string
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
# Splitting Data Set
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import recall_score,precision_score,f1_score
parser = English()

In [2]:
import os
cwd = os.getcwd()
print("Estou na diretoria", cwd)

Estou na diretoria /Users/macbookpro/Documents/pratica-jupter/Text_Mining/Trabalho_1


In [3]:
data=[]
with open("Tweets_EN_sentiment_train.csv", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile )
    for d in reader:
        data.append((d["tweet"],d["text"],d["class"])) 
data

[('1229448571', 'some count sheep, you count cats ', 'pos'),
 ('1230261573',
  "We haven't been to Turning Stone in ages! Have a great time ",
  'pos'),
 ('1228861415', 'What up ', 'pos'),
 ('1228489365', "I'm good ", 'pos'),
 ('1229800235',
  "I know, wonder if USER is going to buy an apartment after he's bought the jets ",
  'pos'),
 ('1230646056', 'ok cool ', 'pos'),
 ('1228964793', "USER Facebook is down. It's not you guys ", 'pos'),
 ('1230178894',
  'Sorry I forgot my &lt;sarcasm&gt; tags. I find I need to use those a lot. ',
  'pos'),
 ('1228887834',
  'Dude. Your stuff on mutiny.in kinda egged me on Hats off',
  'pos'),
 ('1229831457', '1st year of high school  (vmbo, pretty simple :p)', 'pos'),
 ('1229562656',
  "I have to agree with USER Design...I have Vista-the most hated of them all-and haven't had a freeze up in a year ",
  'pos'),
 ('1230682035',
  'lining at the bottom of your bird cage is a good one  USER haha thats thats one thing that i really think CAN be generalise

In [4]:
df = pd.read_csv('Tweets_EN_sentiment_train.csv',delimiter=',')

df_one = pd.get_dummies(df["class"])
#pd.factorize(df['class'])[0]
df_two = pd.concat((df_one, df), axis=1)
df_two = df_two.drop(["neg"], axis=1)
df_result = df_two.rename(columns={"pos": "code"})

In [5]:
df_result


,code,tweet,text,class
0,1,1229448571,"some count sheep, you count cats",pos
1,1,1230261573,We haven't been to Turning Stone in ages! Have...,pos
2,1,1228861415,What up,pos
3,1,1228489365,I'm good,pos
4,1,1229800235,"I know, wonder if USER is going to buy an apar...",pos
...,...,...,...,...
39931,1,1230130325,"LOL- F, please",pos
39932,0,1229250927,I wish I could do that too! But it ain't happe...,neg
39933,1,1228710021,I &lt;3 my boyfriend's family.,pos
39934,0,1228100681,404,neg


In [6]:
texto=[]
for index,item in enumerate(data):
    if index <=100:
        texto.append((item[1]))
    
texto

['some count sheep, you count cats ',
 "We haven't been to Turning Stone in ages! Have a great time ",
 'What up ',
 "I'm good ",
 "I know, wonder if USER is going to buy an apartment after he's bought the jets ",
 'ok cool ',
 "USER Facebook is down. It's not you guys ",
 'Sorry I forgot my &lt;sarcasm&gt; tags. I find I need to use those a lot. ',
 'Dude. Your stuff on mutiny.in kinda egged me on Hats off',
 '1st year of high school  (vmbo, pretty simple :p)',
 "I have to agree with USER Design...I have Vista-the most hated of them all-and haven't had a freeze up in a year ",
 'lining at the bottom of your bird cage is a good one  USER haha thats thats one thing that i really think CAN be generalised!! ;)',
 'URL - "Only buy for your Secret Santa." ... Joyous fail ',
 'i really miss her - I loved her videos/blogs. ',
 'my husband has a deep and irrational fear of chiropractors.    My own gym sucks and has not massage facilities.',
 'No... just heard abt it... still my product uses xm

### I.2 Preparação dos dados e criação de uma baseline

In [7]:
from textblob import TextBlob

In [8]:
train =[]
sentimento= "";
for t in  texto:
    if  TextBlob(t).sentiment.polarity >0:
        sentimento = "Positivo"
        train.append((t,sentimento))
    elif TextBlob(t).sentiment.polarity <0:
        sentimento = "Negativo"
        train.append((t,sentimento))
    else:
        sentimento = "Neutro"
    print(t, "==>",'Sentimento==>',sentimento,"Polaridade==>",TextBlob(t).sentiment.polarity)

some count sheep, you count cats  ==> Sentimento==> Neutro Polaridade==> 0.0
We haven't been to Turning Stone in ages! Have a great time  ==> Sentimento==> Positivo Polaridade==> 0.8
What up  ==> Sentimento==> Neutro Polaridade==> 0.0
I'm good  ==> Sentimento==> Positivo Polaridade==> 0.7
I know, wonder if USER is going to buy an apartment after he's bought the jets  ==> Sentimento==> Neutro Polaridade==> 0.0
ok cool  ==> Sentimento==> Positivo Polaridade==> 0.425
USER Facebook is down. It's not you guys  ==> Sentimento==> Negativo Polaridade==> -0.15555555555555559
Sorry I forgot my &lt;sarcasm&gt; tags. I find I need to use those a lot.  ==> Sentimento==> Negativo Polaridade==> -0.5
Dude. Your stuff on mutiny.in kinda egged me on Hats off ==> Sentimento==> Neutro Polaridade==> 0.0
1st year of high school  (vmbo, pretty simple :p) ==> Sentimento==> Positivo Polaridade==> 0.29000000000000004
I have to agree with USER Design...I have Vista-the most hated of them all-and haven't had a fr

In [9]:
cl = NaiveBayesClassifier(train)

In [10]:
cl.accuracy(train)

0.9594594594594594

In [11]:
#Converter a nossa Lista para o Texto
texto_1 ="".join(texto)
texto_1

'some count sheep, you count cats We haven\'t been to Turning Stone in ages! Have a great time What up I\'m good I know, wonder if USER is going to buy an apartment after he\'s bought the jets ok cool USER Facebook is down. It\'s not you guys Sorry I forgot my &lt;sarcasm&gt; tags. I find I need to use those a lot. Dude. Your stuff on mutiny.in kinda egged me on Hats off1st year of high school  (vmbo, pretty simple :p)I have to agree with USER Design...I have Vista-the most hated of them all-and haven\'t had a freeze up in a year lining at the bottom of your bird cage is a good one  USER haha thats thats one thing that i really think CAN be generalised!! ;)URL - "Only buy for your Secret Santa." ... Joyous fail i really miss her - I loved her videos/blogs. my husband has a deep and irrational fear of chiropractors.    My own gym sucks and has not massage facilities.No... just heard abt it... still my product uses xml good morning. I love fresh snow, all we have here is a lot of mud rob

In [12]:
# tokenizaçao 

single_tokinez = list(map(str.lower,word_tokenize(texto_1)))

print(single_tokinez)

['some', 'count', 'sheep', ',', 'you', 'count', 'cats', 'we', 'have', "n't", 'been', 'to', 'turning', 'stone', 'in', 'ages', '!', 'have', 'a', 'great', 'time', 'what', 'up', 'i', "'m", 'good', 'i', 'know', ',', 'wonder', 'if', 'user', 'is', 'going', 'to', 'buy', 'an', 'apartment', 'after', 'he', "'s", 'bought', 'the', 'jets', 'ok', 'cool', 'user', 'facebook', 'is', 'down', '.', 'it', "'s", 'not', 'you', 'guys', 'sorry', 'i', 'forgot', 'my', '&', 'lt', ';', 'sarcasm', '&', 'gt', ';', 'tags', '.', 'i', 'find', 'i', 'need', 'to', 'use', 'those', 'a', 'lot', '.', 'dude', '.', 'your', 'stuff', 'on', 'mutiny.in', 'kinda', 'egged', 'me', 'on', 'hats', 'off1st', 'year', 'of', 'high', 'school', '(', 'vmbo', ',', 'pretty', 'simple', ':', 'p', ')', 'i', 'have', 'to', 'agree', 'with', 'user', 'design', '...', 'i', 'have', 'vista-the', 'most', 'hated', 'of', 'them', 'all-and', 'have', "n't", 'had', 'a', 'freeze', 'up', 'in', 'a', 'year', 'lining', 'at', 'the', 'bottom', 'of', 'your', 'bird', 'cage'

In [13]:
#removing Stop words
stop_words= []
stop_words = set(stopwords.words('english'))
filtered_word = [word for word in single_tokinez if word not in stop_words]

In [14]:
# Removing Ponctuation 
stop_word_en_punctu = stop_words.union(set(punctuation))
filtered_word = [word for word in single_tokinez if word not in stop_word_en_punctu]

In [15]:
print(filtered_word)

['count', 'sheep', 'count', 'cats', "n't", 'turning', 'stone', 'ages', 'great', 'time', "'m", 'good', 'know', 'wonder', 'user', 'going', 'buy', 'apartment', "'s", 'bought', 'jets', 'ok', 'cool', 'user', 'facebook', "'s", 'guys', 'sorry', 'forgot', 'lt', 'sarcasm', 'gt', 'tags', 'find', 'need', 'use', 'lot', 'dude', 'stuff', 'mutiny.in', 'kinda', 'egged', 'hats', 'off1st', 'year', 'high', 'school', 'vmbo', 'pretty', 'simple', 'p', 'agree', 'user', 'design', '...', 'vista-the', 'hated', 'all-and', "n't", 'freeze', 'year', 'lining', 'bottom', 'bird', 'cage', 'good', 'one', 'user', 'haha', 'thats', 'thats', 'one', 'thing', 'really', 'think', 'generalised', 'url', '``', 'buy', 'secret', 'santa', "''", '...', 'joyous', 'fail', 'really', 'miss', 'loved', 'videos/blogs', 'husband', 'deep', 'irrational', 'fear', 'chiropractors', 'gym', 'sucks', 'massage', 'facilities.no', '...', 'heard', 'abt', '...', 'still', 'product', 'uses', 'xml', 'good', 'morning', 'love', 'fresh', 'snow', 'lot', 'mud', '

#### Analisar as frequencias das palavras 

In [16]:
fdist = FreqDist()
for word in filtered_word:
    fdist[str(word)]+=1
fdist
    

FreqDist({'...': 18, "n't": 13, "'m": 12, 'user': 10, "'s": 10, 'love': 9, 'like': 9, 'good': 8, 'one': 7, 'still': 7, ...})

In [17]:
top_20= fdist.most_common(20)
top_20

[('...', 18),
 ("n't", 13),
 ("'m", 12),
 ('user', 10),
 ("'s", 10),
 ('love', 9),
 ('like', 9),
 ('good', 8),
 ('one', 7),
 ('still', 7),
 ('know', 6),
 ('would', 6),
 ('going', 5),
 ('cool', 5),
 ('oh', 5),
 ('work', 5),
 ('hope', 5),
 ('always', 5),
 ('got', 5),
 ('well', 5)]

### Bibloteca Spacy Para o tratamento de part-of-speech Lemmmetizaçao e Named Entity Recognition

In [18]:
#!python -m spacy download en_core_web_sm

In [19]:
import spacy
import spacy


nlp = spacy.load("en_core_web_sm")


In [20]:
spacy.__version__

'3.1.4'

In [21]:
text_spacy = "".join(texto)

In [22]:
doc = nlp(text_spacy)

In [23]:
for token in doc:
    print(token)

some
count
sheep
,
you
count
cats
We
have
n't
been
to
Turning
Stone
in
ages
!
Have
a
great
time
What
up
I
'm
good
I
know
,
wonder
if
USER
is
going
to
buy
an
apartment
after
he
's
bought
the
jets
ok
cool
USER
Facebook
is
down
.
It
's
not
you
guys
Sorry
I
forgot
my
&
lt;sarcasm&gt
;
tags
.
I
find
I
need
to
use
those
a
lot
.
Dude
.
Your
stuff
on
mutiny.in
kinda
egged
me
on
Hats
off1st
year
of
high
school
 
(
vmbo
,
pretty
simple
:
p)I
have
to
agree
with
USER
Design
...
I
have
Vista
-
the
most
hated
of
them
all
-
and
have
n't
had
a
freeze
up
in
a
year
lining
at
the
bottom
of
your
bird
cage
is
a
good
one
 
USER
haha
that
s
that
s
one
thing
that
i
really
think
CAN
be
generalised
!
!
;)
URL
-
"
Only
buy
for
your
Secret
Santa
.
"
...
Joyous
fail
i
really
miss
her
-
I
loved
her
videos
/
blogs
.
my
husband
has
a
deep
and
irrational
fear
of
chiropractors
.
   
My
own
gym
sucks
and
has
not
massage
facilities
.
No
...
just
heard
abt
it
...
still
my
product
uses
xml
good
morning
.
I
love
fresh
snow


In [24]:
# criar as frases 
token_sent= []
for token in doc.sents:
    
    token_sent.append(token)
    

In [25]:
#  pegar a primeira frase 
token_sent

[some count sheep, you count cats We haven't been to Turning Stone in ages!,
 Have a great time,
 What up,
 I'm good,
 I know, wonder if USER is going to buy an apartment after he's bought the jets ok cool USER Facebook is down.,
 It's not you guys,
 Sorry I forgot my &lt;sarcasm&gt; tags.,
 I find I need to use those a lot.,
 Dude.,
 Your stuff on mutiny.in kinda egged me on Hats off1st year of high school  (vmbo, pretty simple :p)I have to agree with USER Design...,
 I have Vista-the most hated of them all-and haven't had a freeze up in a year lining at the bottom of your bird cage is a good one  USER haha thats thats one thing that i really think CAN be generalised!!,
 ;),
 URL - "Only buy for your Secret Santa." ...,
 Joyous fail i really miss her - I loved her videos/blogs.,
 my husband has a deep and irrational fear of chiropractors.,
    My own gym sucks and has not massage facilities.,
 No... just heard abt it... still my product uses xml good morning.,
 I love fresh snow, all 

In [26]:
frase1 = token_sent[0]

### Part of Speech Tagging 

In [27]:
for token in frase1:
    print(token.text,token.pos_,token.dep_)

some DET det
count NOUN compound
sheep NOUN nsubj
, PUNCT punct
you PRON nsubj
count VERB ccomp
cats NOUN dobj
We PRON nsubj
have AUX aux
n't PART neg
been VERB ROOT
to ADP prep
Turning PROPN compound
Stone PROPN pobj
in ADP prep
ages NOUN pobj
! PUNCT punct


#### Visualizaçao de Part Of speech

In [28]:
from spacy import displacy
displacy.render(frase1, style="dep")

In [29]:
# aplicaçao de lematizaçao 
filtred_word = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in token_sent]

In [ ]:


## Explore the NCR Lexico

#Upload the file
lexicon = pd.read_csv("NCR-lexicon.csv")

#Columns Name
print(lexicon.columns)

#First 9 lines 
lexicon.head(10)

#count the numbers of elements 
len(lexicon)

# Count the numbers of the negative and positives elements in lexico 

number_pos = 0  #when the positive=1 and negative=0
number_neg = 0  #when the positive=0 and negative=1
number_neut_pos = 0 #when the positive=1 and negative=1, can have a positive and a negative side in simultaneos
number_neut = 0 #when the positive=0 and negative=0, dont have positive or a negative sentimental

for i in lexicon.index:
    if lexicon['Positive'][i] == 1 and lexicon['Negative'][i] == 0: #positive
        number_pos = number_pos + 1 
    elif lexicon['Positive'][i] == 0 and lexicon['Negative'][i] == 1: #negative
        number_neg = number_neg + 1
    elif lexicon['Positive'][i] == 1 and lexicon['Negative'][i] == 1: #can have a positive and a negative side in simultaneos
        number_neut_pos = number_neut_pos + 1
    else: #neutral negative
        number_neut = number_neut + 1 #any sentiment

print("Number of positive words: ", number_pos)
print("Number of negative words: ", number_neg)
print("Number of neutral words/without sentiment: ",number_neut )
print("Number of positive and negative words in simultaneous: ",number_neut_pos )

## Count the numbers of the negative and positives elements in lexico 

label = ('Positive', 'Negative', 'Positive and negative(simultaneous)','Without sentiment')
type_sentiment = np.arange(len(label))
numbers = [number_pos, number_neg, number_neut_pos,number_neut]

plt.barh(type_sentiment, numbers, align='center', alpha=1)
plt.yticks(type_sentiment, label)
plt.xlabel('Number of words')
plt.title('Analisys of data - NCR Lexicon')

plt.show()

#Uoload the lexico NCR data set

NCR_Lexicon = []
with open("NCR-Lexicon.csv", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile )
    for d in reader:
        NCR_Lexicon.append((d["English"],d["Positive"],d["Negative"])) 

Tarefas a realizar:
0- Gráfico com percentagens 
1- Léxico sem limpeza de dados e sem negação
2- Léxico com limpeza de dados: letras minusculas, retirar números, retirar simbolos, retirar stop words, retirar carateres especiais sem negação 
3- Léxico com limpeza de dados: letras minusculas, retirar números, retirar simbolos, retirar stop words, retirar carateres especiais E COM TRATAMENTO DA NEGAÇÃO
4 - VER EXEMPLO DO VITOR/LUIS 
5 - Incluir o neutro = [0,0] e posso incluir o positivo e negativo em simultâneos
6 - Accuranrry 

dict = {}
a = {}
for i in lexicon.index:
    if lexicon['Positive'][i] == 1 and lexicon['Negative'][i] == 0: #positive
        #print(lexicon['English'][i],' ', lexicon['Positive'][i], ' ',lexicon['Negative'][i], 'olaa1')
        #print(dict)
        word = lexicon['English'][i] 
        sentiment = 1
        a = {word:sentiment}
    elif lexicon['Positive'][i] == 0 and lexicon['Negative'][i] == 1: #negative
        #print(lexicon['English'][i],' ', lexicon['Positive'][i], ' ',lexicon['Negative'][i], 'olaa2')
        #print(dict)
        word = lexicon['English'][i] 
        sentiment = -1
        a = {word:sentiment}
    dict.update(a)
    


#Numbers with elements in the lexico with positive and negative value or elements present in the dictionary (dict)
len(dict)

# show the elements of the dict
dict

## Create a Lexico classifier - without data cleaning and without treatment of negation - train data set 

a = 0
b = []
result = []
list_sentiment = []

def classifier_1 (tweet_train):
    a = 0
    b = []
    for i in tweet_train:
        for t in i.split():        
            if dict.get(t) == 1 or dict.get(t) == -1:
                print(dict.get(t))
                a = dict.get(t) + a
        result.append(a)
        a = 0
    for i in result:
        if i > 0:
            list_sentiment.append(1)
        else:
            list_sentiment.append(-1)

classifier_1(tweeter_train)

tweters_verify = []
with open("Tweets_EN_sentiment_train.csv", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile )
    for d in reader:
        tweters_verify.append((d["class"])) 


#transform the twetters classification in number
tweters_verify_number = []
for i in tweters_verify:
    if i == 'pos':
        tweters_verify_number.append(1)
    else:
        tweters_verify_number.append(-1)

len(tweters_verify_number)

len(list_sentiment)

#accuracy 
counter = 0
for t in range(len(tweters_verify_number)):
    if tweters_verify_number[t] == list_sentiment[t]:
        counter = counter + 1
        
print('Numbers of right classificatioons: ', counter )   
print('Accuracy: ', counter/len(tweters_verify_number)*100, '%' )   

## Create a Lexico classifier - without data cleaning and without treatment of negation - teste data set 

a_test = 0
b_test = []
result_test = []
list_sentiment_test = []

def classifier_1 (tweet_test):
    a_test = 0
    b_test = []
    for i in tweet_test:
        for t in i.split():        
            if dict.get(t) == 1 or dict.get(t) == -1:
                print(dict.get(t))
                a_test = dict.get(t) + a
        result_test.append(a)
        a_test = 0
    for i in result_test:
        if i > 0:
            list_sentiment_test.append(1)
        else:
            list_sentiment_test.append(-1)

classifier_1(tweeter_test)

tweters_verify_test = []
with open("Tweets_EN_sentiment_test.csv", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile )
    for d in reader:
        tweters_verify_test.append((d["class"])) 

#transform the twetters classification in number
tweters_verify_number_test = []

for i in tweters_verify_test:
    if i == 'pos':
        tweters_verify_number_test.append(1)
    else:
        tweters_verify_number_test.append(-1)

### Named Entity Recognition

In [30]:
entities =[]

for ent in doc.ents:
    print(ent.text,ent.label_)
    entities.append(ent.text)

Turning Stone PERSON
USER ORG
Hats PERSON
USER Design ORG
Vista ORG
a year DATE
Luftwaffemuseum PERSON
German NORP
3 hours TIME
Todd PERSON
Aweber PERSON
Non-Profits ORG
Gatlinburg GPE
Tenn. GPE
Today DATE
4 hours TIME
two days DATE
3 CARDINAL
a week DATE
m&m PERSON
two CARDINAL
m&m PERSON
SLR WORK_OF_ART
USER ORG
tomorrow DATE
U cashmap IM EATING OLD CANDY  AND ITS GOOD.Good ORG
4 CARDINAL
Kimberly PERSON
TEXAS GPE
Nicole PERSON
pHey ORG
PR Day EVENT
these days DATE
days DATE
FaD PERSON
FUSE ORG
tonight TIME
9pmi DATE
about 1pm TIME
last night TIME
today DATE
tomorrow DATE
new year DATE
tonight TIME
g1 CARDINAL
eBay ORG
last year DATE
a rough day DATE
four CARDINAL
today DATE
harry connick PERSON
jr. GPE
Just 8 CARDINAL
400 CARDINAL
2.0 CARDINAL
later this week DATE
Dupa PERSON
seara cu Brian PERSON
ma pregatesc sa ORG
merg ORG
Dreamland  Nighty GPE
Mason ORG
7:30pm-8:30am CARDINAL
yesterday DATE
8:30-6:00 TIME
Jeeze PERSON
LOL ORG
LOL ORG
Carter PERSON
Chuck E. CHeese PERSON
8 CARDIN

In [31]:
displacy.render(doc, style="ent")

##### Manter os Entities com letras maiúsculas e o resto para minuscula 

In [32]:
# 
filtered_entitie=[]
for word in filtered_word:
    if word in entities:
        filtered_entitie.append(word.upper())
    else:
        filtered_entitie.append(word.lower())
        

In [33]:
print(filtered_entitie)

['count', 'sheep', 'count', 'cats', "n't", 'turning', 'stone', 'ages', 'great', 'time', "'m", 'good', 'know', 'wonder', 'user', 'going', 'buy', 'apartment', "'s", 'bought', 'jets', 'ok', 'cool', 'user', 'facebook', "'s", 'guys', 'sorry', 'forgot', 'lt', 'sarcasm', 'gt', 'tags', 'find', 'need', 'use', 'lot', 'dude', 'stuff', 'mutiny.in', 'kinda', 'egged', 'hats', 'off1st', 'year', 'high', 'school', 'vmbo', 'pretty', 'simple', 'p', 'agree', 'user', 'design', '...', 'vista-the', 'hated', 'all-and', "n't", 'freeze', 'year', 'lining', 'bottom', 'bird', 'cage', 'good', 'one', 'user', 'haha', 'thats', 'thats', 'one', 'thing', 'really', 'think', 'generalised', 'url', '``', 'buy', 'secret', 'santa', "''", '...', 'joyous', 'fail', 'really', 'miss', 'loved', 'videos/blogs', 'husband', 'deep', 'irrational', 'fear', 'chiropractors', 'gym', 'sucks', 'massage', 'facilities.no', '...', 'heard', 'abt', '...', 'still', 'product', 'uses', 'xml', 'good', 'morning', 'love', 'fresh', 'snow', 'lot', 'mud', '

In [34]:
len(filtered_entitie)

872

### analisar as frequencias das Palavras

## Machine learning


## Machine Learning With SKlearn

In [35]:
# Vectorization
vectorizer = CountVectorizer(stop_words='english') 
#classifier = LinearSVC()
tfvectorizer = TfidfVectorizer(norm=None)
#feacture_test = vectorizer.fit_transform(df_result.text.values.astype('str'))
all_feactures= vectorizer.fit_transform(df_result.text.values.astype('str'))
#all_feactures= vectorizer.fit_transform(filtered_entitie)

In [36]:
# Using Tfidf
#tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

all_feactures.shape

(39936, 32468)

In [37]:
vectorizer.vocabulary_

{'count': 6791,
 'sheep': 25265,
 'cats': 5150,
 'haven': 12813,
 'turning': 29347,
 'stone': 27020,
 'ages': 1490,
 'great': 12163,
 'time': 28549,
 'good': 11933,
 'know': 15765,
 'wonder': 31648,
 'user': 30080,
 'going': 11904,
 'buy': 4704,
 'apartment': 2117,
 'bought': 4092,
 'jets': 15007,
 'ok': 20087,
 'cool': 6657,
 'facebook': 9966,
 'guys': 12410,
 'sorry': 26345,
 'forgot': 10808,
 'lt': 16978,
 'sarcasm': 24542,
 'gt': 12316,
 'tags': 27758,
 'need': 19196,
 'use': 30071,
 'lot': 16908,
 'dude': 8759,
 'stuff': 27186,
 'mutiny': 18870,
 'kinda': 15639,
 'egged': 9092,
 'hats': 12798,
 '1st': 258,
 'year': 32078,
 'high': 13150,
 'school': 24720,
 'vmbo': 30608,
 'pretty': 22201,
 'simple': 25598,
 'agree': 1505,
 'design': 7842,
 'vista': 30580,
 'hated': 12790,
 'freeze': 10977,
 'lining': 16602,
 'bird': 3600,
 'cage': 4796,
 'haha': 12490,
 'thats': 28268,
 'thing': 28346,
 'really': 23116,
 'think': 28351,
 'generalised': 11547,
 'url': 30048,
 'secret': 24923,
 'san

In [38]:
X_train, X_test, y_train, y_test = train_test_split(all_feactures,df_result.code, test_size=0.3, random_state=47)

In [39]:
X_train.shape

(27955, 32468)

In [40]:
X_test.shape

(11981, 32468)

In [41]:
# Classificaçao com Linear SVC
classifier =LinearSVC()           

In [42]:
classifier.fit(X_train,y_train)

LinearSVC()

In [43]:
nr_correct = (y_test== classifier.predict(X_test)).sum()
nr_incorrect = y_test.size - nr_correct

## Resultado com o Modelo de Teste

In [44]:
model_test = classifier.predict(X_test)

In [45]:
print(" the number of  correct prection is ", nr_correct)
print(" the number of  incorrect prection is ", nr_incorrect)
print("Accuracy: ", metrics.accuracy_score(y_test, model_test))
print("Precision: ", metrics.precision_score(y_test, model_test, average="macro"))
print("Recall: ", metrics.recall_score(y_test, model_test, average="macro"))
print("F1-measure: ", metrics.f1_score(y_test, model_test, average="macro"))

 the number of  correct prection is  9940
 the number of  incorrect prection is  2041
Accuracy:  0.8296469409899007
Precision:  0.6896663772869039
Recall:  0.6447610883275152
F1-measure:  0.661198497992197


In [46]:
for (sample,pred) in zip(token_sent,model_test):
    print(sample,"Prediction=>",pred)

some count sheep, you count cats We haven't been to Turning Stone in ages! Prediction=> 1
Have a great time Prediction=> 0
What up Prediction=> 1
I'm good Prediction=> 1
I know, wonder if USER is going to buy an apartment after he's bought the jets ok cool USER Facebook is down. Prediction=> 1
It's not you guys Prediction=> 1
Sorry I forgot my &lt;sarcasm&gt; tags. Prediction=> 0
I find I need to use those a lot. Prediction=> 1
Dude. Prediction=> 1
Your stuff on mutiny.in kinda egged me on Hats off1st year of high school  (vmbo, pretty simple :p)I have to agree with USER Design... Prediction=> 1
I have Vista-the most hated of them all-and haven't had a freeze up in a year lining at the bottom of your bird cage is a good one  USER haha thats thats one thing that i really think CAN be generalised!! Prediction=> 0
;) Prediction=> 1
URL - "Only buy for your Secret Santa." ... Prediction=> 1
Joyous fail i really miss her - I loved her videos/blogs. Prediction=> 1
my husband has a deep and i